## Import the Training and Test Data

In [3]:
import pandas as pd
from matplotlib import pyplot as plt
import csv
import numpy as np
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import sklearn.metrics
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectFromModel
from xgboost.sklearn import XGBRegressor
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import sys
!{sys.executable} -m pip install xgboost
train = pd.read_csv('/Users/vijayswamy/Outcomes/github-code/Housing_Prices/data/train.csv')
test = pd.read_csv('/Users/vijayswamy/Outcomes/github-code/Housing_Prices/data/test.csv')

thinc 6.11.2 requires hypothesis<3,>=2, which is not installed.
thinc 6.11.2 requires pathlib<2.0.0,>=1.0.0, which is not installed.
spacy 2.0.11 requires pathlib, which is not installed.
smart-open 1.6.0 requires bz2file, which is not installed.
msgpack-numpy 0.4.3 requires msgpack>=0.3.0, which is not installed.
distributed 1.21.8 requires msgpack, which is not installed.
thinc 6.11.2 has requirement msgpack-numpy==0.4.1, but you'll have msgpack-numpy 0.4.3 which is incompatible.
thinc 6.11.2 has requirement msgpack-python==0.5.4, but you'll have msgpack-python 0.5.6 which is incompatible.
spacy 2.0.11 has requirement regex==2017.4.5, but you'll have regex 2018.6.21 which is incompatible.
spacy 2.0.11 has requirement thinc<6.11.0,>=6.10.1, but you'll have thinc 6.11.2 which is incompatible.
ipywidgets 7.1.0 has requirement widgetsnbextension~=3.1.0, but you'll have widgetsnbextension 3.0.2 which is incompatible.
Cache entry deserialization failed, entry ignored
You are using pip vers


## Numerical Features

Feature Engineering

In [4]:
#Convert neighborhoods into numerical codes for feature engineering
train["Neighborhood"] = pd.Categorical(train["Neighborhood"]).codes
train["Neighborhood"] = train["Neighborhood"].astype(int)

test["Neighborhood"] = pd.Categorical(test["Neighborhood"]).codes
test["Neighborhood"] = test["Neighborhood"].astype(int)

#Converted garage quality and conditions into numerical codes for feature engineering
train["Garage Qual"] = pd.Categorical(train["Garage Qual"]).codes
train["Garage Qual"] = train["Garage Qual"].astype(int)

test["Garage Qual"] = pd.Categorical(test["Garage Qual"]).codes
test["Garage Qual"] = test["Garage Qual"].astype(int)

train["Garage Cond"] = pd.Categorical(train["Garage Cond"]).codes
train["Garage Cond"] = train["Garage Cond"].astype(int)

test["Garage Cond"] = pd.Categorical(test["Garage Cond"]).codes
test["Garage Cond"] = test["Garage Cond"].astype(int)

#Basement Quality and Condition
train["Bsmt Qual"] = pd.Categorical(train["Bsmt Qual"]).codes
train["Bsmt Qual"] = train["Bsmt Qual"].astype(int)

test["Bsmt Qual"] = pd.Categorical(test["Bsmt Qual"]).codes
test["Bsmt Qual"] = test["Bsmt Qual"].astype(int)

train["Bsmt Cond"] = pd.Categorical(train["Bsmt Cond"]).codes
train["Bsmt Cond"] = train["Bsmt Cond"].astype(int)

test["Bsmt Cond"] = pd.Categorical(test["Bsmt Cond"]).codes
test["Bsmt Cond"] = test["Bsmt Cond"].astype(int)

#Basement Exposure
train["Bsmt Exposure"] = pd.Categorical(train["Bsmt Exposure"]).codes
train["Bsmt Exposure"] = train["Bsmt Exposure"].astype(int)

test["Bsmt Exposure"] = pd.Categorical(test["Bsmt Exposure"]).codes
test["Bsmt Exposure"] = test["Bsmt Exposure"].astype(int)

#Kitchen Quality
train["Kitchen Qual"] = pd.Categorical(train["Kitchen Qual"]).codes
train["Kitchen Qual"] = train["Kitchen Qual"].astype(int)

test["Kitchen Qual"] = pd.Categorical(test["Kitchen Qual"]).codes
test["Kitchen Qual"] = test["Kitchen Qual"].astype(int)

test["Kitchen AbvGr"] = pd.Categorical(test["Kitchen AbvGr"]).astype(int)

#Fireplace Quality
train["Fireplace Qu"] = pd.Categorical(train["Fireplace Qu"]).codes
train["Fireplace Qu"] = train["Fireplace Qu"].astype(int)

test["Fireplace Qu"] = pd.Categorical(test["Fireplace Qu"]).codes
test["Fireplace Qu"] = test["Fireplace Qu"].astype(int)

#Pool Quality
train["Pool QC"] = pd.Categorical(train["Pool QC"]).codes
train["Pool QC"] = train["Pool QC"].astype(float)

test["Pool QC"] = pd.Categorical(test["Pool QC"]).codes
test["Pool QC"] = test["Pool QC"].astype(float)


#Total all the Square Feet 
train["Total_SF"] = train["Total Bsmt SF"] + train["1st Flr SF"] + train["2nd Flr SF"]
test["Total_SF"] = train["Total Bsmt SF"] + test["1st Flr SF"] + test["2nd Flr SF"]

train["Lot Frontage"] = train["Lot Frontage"]/140
test["Lot Frontage"] = test["Lot Frontage"]/140

#Calculated Overall Grade of the property based upon the quality and condition
train["OverallGrade"] = train["Overall Qual"] * train["Overall Cond"]
test["OverallGrade"] = test["Overall Qual"] * test["Overall Cond"]

#Calculated total square feet of porch
train["TotalPorchSF"] = train["Open Porch SF"] + train["Enclosed Porch"] + train["3Ssn Porch"] + train["Screen Porch"]
test["TotalPorchSF"] = test["Open Porch SF"] + test["Enclosed Porch"] + test["3Ssn Porch"] + test["Screen Porch"]

#Calculated how good the garage is by its quality and condition
train["GarageGrade"] = train["Garage Qual"] * train["Garage Cond"]
test["GarageGrade"] = test["Garage Qual"] * test["Garage Cond"]

#Calculated how good the basement is
train["BasementGrade"] = train["Bsmt Qual"] * train["Bsmt Cond"] 
test["BasementGrade"] = test["Bsmt Qual"] * test["Bsmt Cond"]

#Calculated how good the fireplace is
train["FireplaceScore"] = train["Fireplaces"] * train["Fireplace Qu"]
test["FireplaceScore"] = test["Fireplaces"] * test["Fireplace Qu"]

train["KitchenGrade"] = train["Kitchen Qual"] * train["Kitchen AbvGr"]
test["KitchenGrade"] = test["Kitchen Qual"] * test["Kitchen AbvGr"]


In [5]:
numerical_features = list(train._get_numeric_data().drop(["Id", "PID", "SalePrice", "MS SubClass", "Mo Sold"], axis=1))

Remove the outliers from the data. Outliers are any values that are greater than 3 standard deviations from the mean. 

In [6]:
# #Remove the outliers on both ends of the scale
for feature in numerical_features:
    train[feature].fillna(train[feature].mean(), inplace = True)
    test[feature].fillna(test[feature].mean(), inplace = True)
    train[np.abs(train[feature]-train[feature].mean()) <= (3*train[feature].std())]
    test[np.abs(test[feature]-test[feature].mean()) <= (3*test[feature].std())]
    train[np.abs(train[feature]-train[feature].mean()) >= (3*train[feature].std())]
    test[np.abs(test[feature]-test[feature].mean()) >= (3*test[feature].std())]

Use the XGBRegressor and the fine tune the model with the maximum alpha to ensure better performance. 

In [62]:
#to find the max log alpha using logspace
params = {}
alphas = np.logspace(5,8, num = 10)
params = {'reg_alpha': alphas}

#input the max alpha into the grid search cv 
gs = GridSearchCV(XGBRegressor(learning_rate=0.08,max_depth=6,n_estimators=120, colsample_bytree = 0.35, verbose=1, gamma=5), param_grid = params)
gs.fit(train[numerical_features], train["SalePrice"])
print(gs.best_score_)
print(gs.best_params_)

0.9176241154708668
{'reg_alpha': 100000.0}


## Categorical Data

In [7]:
categorical_columns = list(train.select_dtypes(include=["object"]).columns)
for col in categorical_columns:
    train[col].fillna("N/A", inplace = True)
    test[col].fillna("N/A", inplace = True)
    train_values = sorted(list(train[col].unique()))
    test_values = sorted(list(test[col].unique()))
    categories = set(train_values + test_values)
    test[col] = pd.Categorical(test[col], categories = categories)
    train[col] = pd.Categorical(train[col], categories = categories)

In [8]:
train_dummies = pd.get_dummies(train[categorical_columns])
test_dummies = pd.get_dummies(test[categorical_columns])

In [55]:
X_train = pd.concat([train_dummies, train[numerical_features]], axis = 1)
X_test = pd.concat([test_dummies, test[numerical_features]], axis = 1)

## Model

After we add the categorical features into our model, we want to once again use XGBRegressor and optimize this algorithm by finding the maximum alpha. 

In [63]:
#to find the max log alpha using logspace

alphas = np.logspace(5,8, num = 10)
params = {'reg_alpha': alphas}

#http://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection

#input the max alpha into the grid search cv 
gs = GridSearchCV(XGBRegressor(learning_rate=0.1,max_depth=3,n_estimators=1500, colsample_bytree = 0.35, gamma=5,early_stopping_rounds=5), param_grid = params)
gs.fit(X_train, train["SalePrice"])
print(gs.best_score_)
print(gs.best_params_)

0.9204215653275377
{'reg_alpha': 100000.0}


We now use our model to predict housing prices.

In [64]:
pred = gs.predict(X_test)

### Submission to Kaggle

This will write two rows of data to the submission.csv file: One which is the ID and the other is the predicted housing price. 

In [65]:
test["SalePrice"] = pred

In [66]:
test[['Id','SalePrice']].to_csv('submission.csv', index = False)